<a href="https://colab.research.google.com/github/nganngants/Triton_AITempoRun_Information_Retrieval/blob/main/%5BTempoRun%5D_Triton_Information_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download data

In [ ]:
import json
#Download lyrics
!gdown --id 1V7WxGaVv3UamD4AM5qnqDNzkBSw6paQy
#Download queries
!gdown --id 1srNAjKMTncZUgxDTZZHfMnTbPd9ic2xx

Downloading...
From: https://drive.google.com/uc?id=1V7WxGaVv3UamD4AM5qnqDNzkBSw6paQy
To: /content/2335_Songs.json
100% 4.33M/4.33M [00:00<00:00, 186MB/s]
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1srNAjKMTncZUgxDTZZHfMnTbPd9ic2xx 



# Preliminaries & Corpus Creation

In [ ]:
!pip install -q tf-models-official==2.3.0

In [ ]:
# Import required libraries
import os
import pprint as pp
import numpy as np
import json
import tensorflow as tf 
from gensim.corpora import WikiCorpus

from official.modeling import tf_utils
from official import nlp
from official.nlp import bert

# Load the required submodules
import official.nlp.optimization
import official.nlp.bert.bert_models
import official.nlp.bert.configs
import official.nlp.bert.run_classifier
import official.nlp.bert.tokenization
import official.nlp.data.classifier_data_lib
import official.nlp.modeling.losses
import official.nlp.modeling.models
import official.nlp.modeling.networks

In [ ]:
from tqdm import tqdm
# link to data
CORPUS_SONG_JSON = "2335_Songs.json"
QUERIES_JSON = "500_queries.json"

In [ ]:
with open(CORPUS_SONG_JSON, 'r', encoding='utf-8') as fo:
  corpus_data = json.load(fo)

with open(QUERIES_JSON, 'r', encoding='utf-8') as fo:
  queries_test = json.load(fo)

In [ ]:
import re
import string
!pip3 install vncorenlp

#remove lower case, punctuation, stopwword


In [ ]:
!pip install unidecode

In [ ]:
import unidecode

In [ ]:
def remove_accent(text):
    return unidecode.unidecode(text)

In [ ]:
phrases = ['\[', '\]', 'verse', 'ver', 'ver1', 'ver2', 'chorus', 'pre-chorus', 'hook'];
def normalize_text(lyric):
      lyric = lyric.lower()
      lyric = lyric.replace('\n', ' ')
      lyric = lyric.translate({ord('\n'): None})
      lyric = lyric.translate(str.maketrans("","",string.punctuation))
      for phr in phrases:
        lyric = re.sub(phr,' ', lyric)
      lyric = re.sub('1', 'mot',lyric)
      lyric = re.sub('2', 'hai',lyric)
      lyric = re.sub(' k ', 'khong', lyric)
      lyric = remove_accent(lyric)
      return lyric

In [ ]:
def read_corpus(corpus_data):
 
    # Initialize empty corpus list
    corpus = []
    
    # For each file in the corpus directory, do...
    print("Reading local corpus, please wait...")

    cnt = 0
    for song in corpus_data:
      if song["lyrics"].split("\n")[0].startswith("Bài hát: "):
          lyric = "\n".join(song["lyrics"].split("\n")[1:])
      else:
          lyric = song["lyrics"]
      lyric = normalize_text(lyric)
      corpus.append([song["song_id"], lyric])
        
    # Done, return
    print("Done!")
    return corpus

In [ ]:
# Read corpus! 

corpus = read_corpus(corpus_data)

# Look at some example...



# For some reason (only in Google Colab), this cell might need to be stopped
# for the first run, then run again. 

Reading local corpus, please wait...
Done!


In [ ]:
corpus[0:1000]

[['0000',
  'co phai la duyen khi anh gap em tu kiep truoc the uoc chung doi cung han hoan nhin nhau dam duoi long quyen luyen chuyen mai khong thoi em tu noi dau da den noi nay anh han hoan trao em vong tay va doi ta song chung hanh phuc cung xay dap cho moi duyen nay em oi em oi du phong ba du bao to du nang chay anh van yeu em mai mai khong thoi anh oi anh oi du gian nan du khon kho ta van yeu nhau den cuoi cuoc doi ta van ben nhau qua bao buon vui ta yeu nhau tinh yeu dam say tinh yeu ta dep tinh yeu tham thiet hanh phuc tram nam tinh yeu thuy chung   '],
 ['0001',
  'tu ngay con ra di cha long buon thuong nho co don va sau muon bao ngay thang mong cho tu ngay con ra di toi thoi gian bay gio tham thoat muoi hai nam chan cha thuong dung ngoi ngan ngo dk  me con khong con nua di roi quen tu gia cha mot minh thui thui lanh gia va don coi nay con o phuong xa dung quen noi que nha van con nguoi trong doi nguoi ay la cha gia cha cho con trong hap hoi la thu cuoi cung con oi thu nay mong 

#Matching

In [ ]:
corpus[3][1]


'can ve chieu nao lang le than gay guc xuong don dau  gio thoi gao dem buon buong xuong  the nhan chim trong muon toi loi  long con da thong hoi ngai oi  da bao lan tim chua don dau  bao nhieu lan trot loi nguyen the  ao vong tran gian mai me di tim  dk yeu con chi vi yeu con chua da hy sinh  yeu con chi vi yeu con chua da quen than  quen toi loi trong doi con loi pham  can ve chieu buon nam xua than chua bo vo  bao nhieu lan don roi dau than xac e che  phut cuoi cung tren thanh gia kia  chua guc dau van nho thuong con  chua van tham thu tha toi con'

In [ ]:
def count_match (text, pattern):
  txt = text.split()
  pat = pattern.split()
  #print(txt)
  #print(pat)
  M = len(txt)
  N = len(pat)
  ret = 0
  max = 0.0
  for i in range(0,M - N + 1):
    if (txt[i] == pat[0]):
      count = 0
      k = 0
      for j in range(0,N):
        if i + j + k < M:
          if txt[i + j + k] == pat[j]: count += 1
          else:
            nk = 0
            while i + j + k + nk < M and nk < 4 and txt[i + j + k + nk] != pat[j]: nk += 1
            if i + j + k + nk < M and txt[i + j + k + nk] == pat[j]: 
              k += nk
              count += 1
      if count >= 0.7 * N: ret += 1
      if (max < count / N): max = count / N
  return (ret, max)

In [ ]:
#test
count_match(corpus[0][1], 'ta yeu nhau')

(2, 1.0)

In [ ]:
def get_top_results (re, num = 1):
  re.sort(reverse = True)
  #print(re[0])
  return re[:num]


In [ ]:
#do queries

import csv
import re
with open("submission.csv","w") as csv_writer:
  fieldnames = ['query_id', 'song_id']
  writer = csv.DictWriter(csv_writer, fieldnames=fieldnames)
  writer.writeheader()
  for ques in tqdm(queries_test):
    result = []
    preprocess_query = " ".join(ques["query"].lower().replace("/", "").strip("").split())
    preprocess_query = normalize_text(preprocess_query)
    #print(preprocess_query)
    for song in corpus:
      num_match = count_match(song[1],preprocess_query)
      result.append([num_match, song[0]])
    #print(result)
    sorted_result = get_top_results(result,num = 10)
    #print(sorted_result[0:3])
    final_results = ""
    for item in sorted_result:
      final_results += item[1] + " "
    writer.writerow({'query_id': ques["query_id"], "song_id": "".join(final_results)})


100%|██████████| 500/500 [01:25<00:00,  5.84it/s]
